# LiNGAM: Testing its Assumptions

In this notebook, we will try to infer cause and effect from synthetic data using LiNGAM. The goal is to evaluate the method when its assumptions hold and when they do not. Try to be creative in the latter case, but **do not forget to test linear models with a Gaussian distributed cause and noise**.

## Data Generation & Inspection

Example data generation for a linear causal model with uniformly distributed cause X, noise N, and effect Y---a setting in which LiNGAM should output the correct direction.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# The cause X is non-Gaussian distributed
X = np.random.uniform(size=500)
# The noise N is non-Gaussian distributed
N = np.random.uniform(size=500)
# The effect Y is generated as a linear function
# of it's cause(s) plus an additive noise term
Y = 1.2 * X + N

# Plotting
plt.scatter(X,Y)
plt.xlabel('Cause')
plt.ylabel('Effect')
plt.title('LiNGAM Model')
plt.show()

## LiNGAM Implementation 
Note that this is one possible implementation that uses [HSIC](https://link.springer.com/chapter/10.1007/11564089_7), a kernel independence test, to assess the dependence between potential cause and residuals.

In [ ]:
from sklearn.linear_model import LinearRegression
from cdt.independence.stats import NormalizedHSIC

# Output:
# (causal direction, HSIC score for X->Y, HSIC score for Y->X)
# @causal direction (output 1) is equal to
#      1 if predicted DAG is X -> Y
#     -1 if predicted DAG is Y -> X
def SimpleLiNGAM(X,Y):
    # Fit X -> Y
    regXtoY = LinearRegression().fit(X.reshape(-1,1), Y)
    yEst = regXtoY.predict(X.reshape(-1,1))
    residualsXtoY = (Y - yEst)
    # Fit Y -> X
    regYtoX = LinearRegression().fit(Y.reshape(-1,1), X)
    xEst = regYtoX.predict(Y.reshape(-1,1))
    residualsYtoX = (X - xEst)
    # Use normalized HSIC (a kernal dependence measure; see documentation)
    # to measure the dependence between potential cause and residuals.
    # The smaller the output the more evidence we found for independence.
    test = NormalizedHSIC()
    # Test LiNGAM hypothesis for X -> Y
    hsicXtoY = test.predict(X, residualsXtoY)
    # Test LiNGAM hypothesis for Y -> X
    hsicYtoX = test.predict(Y, residualsYtoX)
    # If hsicXtoY < hsicYtoX, X -> Y is the more likely causal model under LiNGAM.
    if hsicXtoY < hsicYtoX:
        return 1, hsicXtoY, hsicYtoX
    else:
        return -1, hsicXtoY, hsicYtoX

## Let's apply LiNGAM!

Here we apply <code>SimpleLiNGAM</code> to the data we generated above. If the first output of <code>SimpleLiNGAM</code> is equal to 1, it returns the correct direction (X causes Y). 

In [ ]:
SimpleLiNGAM(X,Y)

## Now is your turn! 

Evaluate the performance of LiNGAM for different settings. Try, e.g., to generate the data with <code>np.random.normal(size=XXX)</code>, use non-linear data, or change the sample size. Report your results in a table, and let's discuss them.

The code below is an example evaluation of the data generating process we considered above.

In [ ]:
# Make your results reproducible by setting the seed of the random generator!
np.random.seed(1)
correctInferences = 0
numberOfExperiments = 100
sampleSize = 500
slope = 1.2
for i in range(numberOfExperiments):
    # To keep it simple, let X be the cause, N the noise and Y the effect;
    # s.t. X -> Y is the ground truth model. 
    ## MODIFY HERE
    X = np.random.uniform(size=sampleSize)
    N = np.random.uniform(size=sampleSize)
    Y = slope * X + N
    ## END MODIFY
    (causalDirection, hsicXtoY, hsicYtoX) = SimpleLiNGAM(X,Y)
    if causalDirection == 1:
        # the decision was correct
        correctInferences = correctInferences + 1
        
print(f"Accuracy of LiNGAM: {correctInferences/numberOfExperiments}")